In [3]:
from sympy.solvers import solve
from sympy import *
import numpy as np

init_printing()

# Fonctions utiles 

In [4]:
def convert_MatrixSymbol_to_Array(Matrix, n, m):
    Tableau = []
    for i in range(n):
        Liste = []
        for j in range(m):
            Liste.append(Matrix[i,j])
        Tableau.append(Liste)
    return np.array(Tableau)

def convert_Array_to_vector(Array):
    Vector = []
    for i in range(3):
        Vector.append(Array[i,0])
    return Vector

def convert_Vector_to_array(Vector):
    Tableau = []
    for i in range(3):
        Tableau.append([Vector[i]])
    return np.array(Tableau)


In [5]:
sp      = 4             # Nombre de surface portante
nb_moteur = 4           # Nombre de rotor
nb_tilt = 2             # Nombre de cervomoteur
cp_i    = []            # Liste des surfaces portantes
vB_i    = []            # Vitesse de chaque surface
n_i     = []            # Vecteur associé à une surface portante
N_i     = []            # Dénominateur

## Init Paramètres

In [6]:
A       = MatrixSymbol('A',1,sp)        # Coefficient
v       = MatrixSymbol('v',1,sp)        # Vitesse (??)
Omega   = MatrixSymbol('Omega',3,1)     # Vecteur de rotation
vB      = MatrixSymbol('v_B',3,1)        # Vitesse du corp (drone)
alpha   = MatrixSymbol('alpha',1,sp)    # Coeff alpha
cl_1_fp = Symbol('cl_1_fp')             # Coeff rechercher
cd_0_sa = Symbol('cd_0_sa')             # Coeff rechercher
cd_1_sa = Symbol('cd_1_sa}')             # Coeff rechercher
alpha_0 = Symbol('alpha_0')             # Coeff rechercher
delta_t = Symbol('delta_{tilt}', real=True)  # Tilt des moteurs 

In [7]:
Omega   = convert_MatrixSymbol_to_Array(Omega, 3, 1)
vB      = convert_MatrixSymbol_to_Array(vB, 3, 1)
A       = convert_MatrixSymbol_to_Array(A, 1, sp)


array([[Omega[0, 0]],
       [Omega[1, 0]],
       [Omega[2, 0]]], dtype=object)

# Forces Aéro 

In [19]:
def equation_aero(H1,H2,H3,H4):
    Somme_L = np.zeros((3,1))               # Somme des efforts de portance
    Somme_D = np.zeros((3,1))               # Somme des efforts de trainée
    Somme   = np.zeros((3,1))               # Somme des efforts Aéro
    Force_Moteur =np.zeros((nb_moteur,1))   # Poussé des moteurs
    Moment  = np.zeros((3,1))               # Somme des Moments
    
    for i in range(sp):
        cp_i.append(convert_MatrixSymbol_to_Array(MatrixSymbol('cp_%s'%i,3,1),3,1))
        vB_i.append(convert_MatrixSymbol_to_Array(MatrixSymbol('vB_%s'%i,3,1),3,1))
        n_i.append(convert_MatrixSymbol_to_Array(MatrixSymbol('n_%s'%i,3,1),3,1))
        N_i.append(Symbol('N_%s'%i))
                    
        ##### CONDITIONS  PERMETTANTS DE SIMPLIFIER #####
        if(H1): cp_i[i][2,0] = 0                        #
        if(i==1 and H2): A[0,1] = A[0,0]                #
        if(i==1 and H3): cp_i[1][1,0]= -cp_i[0][1,0]    #    
        if(i==3 and H4): cp_i[3][0,0]= cp_i[2][0,0]     #
        #################################################
        
        Prod_vec = convert_Vector_to_array(np.cross(convert_Array_to_vector(cp_i[i]),convert_Array_to_vector(Omega)))
        NumL = vB - np.cross(np.transpose(n_i[i]),np.cross(np.transpose(cp_i[i]), np.transpose(Omega)))
        NumD = vB-Prod_vec-np.dot((vB-Prod_vec).T,n_i[i])*n_i[i]
        
        Li = A[0,i]*v[0,i]**2*(0.5*cl_1_fp*sin(2*(alpha[0,i]+alpha_0)))*(NumL/N_i[i])
        Di = A[0,i]*v[0,i]**2*(cd_0_sa+cd_1_sa*sin(alpha[0,i]+alpha_0)**2)*(NumD/N_i[i])
        
        Somme_L = Somme_L + Li
        Somme_D = Somme_D + Di
        Moment  = Moment  + convert_Vector_to_array(np.cross(convert_Array_to_vector(cp_i[i]),convert_Array_to_vector(Li+Di)))
        
    Somme = -Somme_L-Somme_D
    
    return Somme,Moment

## Poussé des moteurs 

In [ ]:
def pousse_moteur(omega, delta_t)

In [20]:
# Initialisation des hypothèses
H1 = True       # cpi,3 = 0
H2 = True       # A1 = A2
H3 = True       # cp2,2 = -cp1,2
H4 = True       # cp4,1 = cp3,1

Somme,Moment = equation_aero(H1,H2,H3,H4)
#Somme = simplify(Somme)
#Moment= simplify(Moment)
#print(Somme[0][0])
#print(Moment)

# Impression sous forme LaTeX
#print(latex(Somme[0][0]))